#### Code to download PV data.

Updated: November 21, 2024

lol??

In [2]:
#import statements... I think these are all of the relevant ones to what I am doing here. 
import datetime as dt
import numpy as np
import xarray as xr
import math
import netCDF4
import os
import scipy.stats
import pickle
import pandas as pd

/knight/anaconda_jan21/envs/aug21/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
##definition statement for unpacking files
def pvfile(year, month):
    file = xr.open_dataset("/nfs/home11/grad/2020/ef935217/efernandez/Research/Dissertation/era5/pv/era5_an_pv_reg2_6h_"+year+month+".nc")
    daily = file.resample(time='1D').mean() ##convert to 0Z only
    daily['time'] = daily['time'].dt.date ##remove time from data
    pvfile = daily.loc[dict(plev=10000,lat=slice(90,60))] ##retun xarray of desired data, PV over cap at 10hPa 

    return pvfile

In [5]:
##establish month list
months = ['10', '11', '12', '01', '02', '03']

In [8]:
#create empty arrays for the years of data
pv = np.empty((63,182,16,180))
pv[:] = np.nan

In [9]:
#loop years (1959 to 2021)
for year_index in range(63):
    year = str(1959 + year_index)
    print("start:",year)
    #loop months
    for month_index, month in enumerate(months):
        #day range based on month
        if month == '10':  # 10/1 to 10/31
            day_range = range(1, 32)
            day_index = range(0, 31)

            #upload appropriate pv data
            pv_data = pvfile(year, month)
            
        elif month == '11':  #11/1 to 11/30
            day_range = range(1, 31)
            day_index = range(31, 61)
            #upload appropriate pv data
            pv_data = pvfile(year, month)
            
        elif month == '12':  #12/1 to 12/31
            day_range = range(1, 32)
            day_index = range(61, 92)
            #upload appropriate pv data
            pv_data = pvfile(year, month)
            
        elif month == '01':  #1/1 to 1/31
            year = str(int(year)+1)
            print("add year:",year)
            day_range = range(1, 32)
            day_index = range(92, 123)
            #upload appropriate pv data
            pv_data = pvfile(year, month)
            
        elif month == '02':  # February 1st to February 28th (29th on leap years)
            print("check february:",year)
            if year == str(1960) or year == str(1964) or year == str(1968) or year == str(1972) or year == str(1976) or year == str(1980) or year == str(1984) or year == str(1988) or year == str(1992) or year == str(1996) or year == str(2000) or year == str(2004) or year == str(2008) or year == str(2012) or year == str(2016) or year == str(2020):
                day_range = range(1, 29)
                day_index = range(123, 151)
                #upload appropriate pv data
                pv_data = pvfile(year, month)
                
            else:
                day_range = range(1, 28)
                day_index = range(123, 150)
                #upload appropriate pv data
                pv_data = pvfile(year, month)
                
        elif month == '03':  # March 1st to March 31st
            day_range = range(1, 32)
            day_index = range(151, 182)
            #upload appropriate pv data
            pv_data = pvfile(year, month)
        
        #loop through days and save the data
        for day in day_range:
            date = dt.date(int(year), int(month), day)
            # Adjust the day index and save the data
            pv[year_index, day_index[day-1], :, :] = pv_data.sel(time=date)['var60'].values
    
    print("##################")


start: 1959
add year: 1960
check february: 1960
##################
start: 1960
add year: 1961
check february: 1961
##################
start: 1961
add year: 1962
check february: 1962
##################
start: 1962
add year: 1963
check february: 1963
##################
start: 1963
add year: 1964
check february: 1964
##################
start: 1964
add year: 1965
check february: 1965
##################
start: 1965
add year: 1966
check february: 1966
##################
start: 1966
add year: 1967
check february: 1967
##################
start: 1967
add year: 1968
check february: 1968
##################
start: 1968
add year: 1969
check february: 1969
##################
start: 1969
add year: 1970
check february: 1970
##################
start: 1970
add year: 1971
check february: 1971
##################
start: 1971
add year: 1972
check february: 1972
##################
start: 1972
add year: 1973
check february: 1973
##################
start: 1973
add year: 1974
check february: 1974
##############

In [10]:
##need to multiply the array by 10**6 to get in PVU
pvu = pv[:,:,:,:]*(1*10**6)

In [11]:
print(pvu.shape)
print(pvu[20,:,0,0])

(63, 182, 16, 180)
[15.17010242 14.94508615 15.93010529 15.74471025 16.35928493 15.73908594
 16.1836906  15.87293809 16.46946839 17.13061647 15.57237192 15.76884824
 16.52610081 17.13759411 17.63387263 17.10432844 16.68080949 17.61154999
 16.35549779 16.88630437 18.15420183 16.60921043 16.97838525 15.96432412
 15.45844862 15.44399856 15.26600317 16.18597889 16.50865306 16.31151827
 14.9828229  15.87685256 14.22073547 14.275578   15.48005093 15.40767698
 15.16955672 15.20897422 15.03054227 15.18499994 15.00584403 14.22402056
 14.13533028 14.41281347 14.17035674 15.97017763 15.91018008 15.09247886
 14.52504875 15.77306102 14.01475311 14.75859608 17.25167022 15.6382448
 16.13515269 18.87812687 16.20491093 16.38221875 17.90578244 15.76563591
 17.52753087 15.03347085 18.40890764 16.7800099  14.42904249 14.06366209
 15.32580427 15.59243901 14.98767961 13.49838931 13.13598113 13.43599797
 14.28105315 13.33082764 13.40540621 16.38847971 14.39823245 14.67345282
 14.1411183  14.22785135 13.91270

In [12]:
pickle.dump(pvu, open("cap_pv100hPa.p", 'wb'))